In [ ]:
'''
Main idea:
- Import relevant libraries, read and create relevant data frames, create new columns based on variables that are 
needed 
- Variables that we calculate: growth, assets, num_years, 
- Then, we use these information to estimate an intrinsic value range

Excel sheet:
- Should include: Cash Flows, assets, market capital, debt 

Calculations:
- PE ratio, Cash flow growth, market cap growth, discounted cash flows and terminal price of stock 


'''

In [1]:
# Import the relevant libraries / Packages 
import math
import pandas as pd
import numpy as np

In [2]:
# Planning:
# Problem now is that: Need to work out where these variables come from. Need a systematic way of choosing the right numeric 
# values for each of the variables



# Other quantitiative measures that aren't included yet:
# Book value and capital ratio 
# What is the PE (price to earnings ratio)

# Do we use revenue, FCF, earnings per share (EPS) or equity?

In [3]:
# Quantitative measures 

# This section of code asks the user the degree of conservativenes

degree = input("How conservative? conservative, average, not conversative\n")
if degree == "conservative":
    print("choice: conservative")
elif degree == "average":
    print("choice: average")
elif degree == "not conservative":
    print("choice: not conservative")
else:
    print("choice: Undefined. Going with manual definition")
    degree = "manual"

How conservative? conservative, average, not conversative
conservative
choice: conservative


In [4]:
# Read the excel file, create dataframes, and do some sanity checks


# Read the excel file and store into a dataframe
df = pd.read_excel('company data.xlsx')

# Create 2 dataframes: One for Capital, CFS, and assets. The other for growth rate calculation
df1 = df[['Cash Flows', 'Capital']]
df2 = df[['Company', 'Start', 'End']].dropna(axis = 0)
df3 = df[['Assets']].fillna(0)

# Get the minimum, max and average cash flows 
cf_min = df["Cash Flows"].min()
cf_avg = df["Cash Flows"].mean()
cf_max = df["Cash Flows"].max()

# Get Total year range 
year_begin = df["Year"].min()
year_end = df["Year"].max()
num_years = df["Year"].count()

# Get Capital data 
cap_avg = df["Capital"].mean()
cap_min = df["Capital"].min()

# CF to Capital selling multiple 
df["Selling Multiple"] = df["Capital"] / df["Cash Flows"]
min_selling_mult = df["Selling Multiple"].min()
avg_selling_mult = df["Selling Multiple"].mean()

### Just some checks: What dataframe looks like and some values
##print(df) 
##print(df1)
##print(df2)
##print(df3)
##
### Sanity check for the values:
##print(cf_min, cf_avg, cf_max) 
##print(year_begin, year_end, num_years)
##print(cap_avg)

   Year  Cash Flows  Assets  Capital   Company     Start       End  \
0  2010       53.09    67.0     5300      Year  2006.000  2016.000   
1  2011       58.00     NaN     5500  Revenue     19.315   215.639   
2  2012       64.00     NaN     6000       EPS     2.270     8.310   
3  2013       70.00     NaN     5000    Equity     9.984   128.249   
4  2014       77.00     NaN     6000       FCF     1.563    53.090   
5  2015       80.00     NaN     5800       NaN       NaN       NaN   
6  2016       80.00     NaN     5800       NaN       NaN       NaN   

   Selling Multiple  
0         99.830477  
1         94.827586  
2         93.750000  
3         71.428571  
4         77.922078  
5         72.500000  
6         72.500000  
   Cash Flows  Capital
0       53.09     5300
1       58.00     5500
2       64.00     6000
3       70.00     5000
4       77.00     6000
5       80.00     5800
6       80.00     5800
    Company     Start       End
0      Year  2006.000  2016.000
1  Revenue     

In [5]:
# Helper functions used for DCF analysis


# Check that the variable can be converted into a float format (Function returns true if variable is numeric convertible)
def check_numeric(variable):
    try:
        variable = float(variable)
        return True
    except ValueError:
        print("Invalid input, taking standard rate\n")
        
        
# Function that calculates the annual growth of return (in decimal) using final price, initial price and number of years 
def annual_growth(init_price, fin_price, num_years):
    ratio_year = pow((fin_price / init_price), 1 / num_years)
    return (ratio_year - 1)

# Function that returns the minimum value from a list 
# Planning to use this for finding the minimum growth rate for intrinsic value calculations
def min_value(growth_list):
    if growth_list:
        return min(growth_list)
    else: 
        print("The list is empty\n")
    return False

# Function that asks for user input (questions include only discount rate and margin of safety)
def ask_input(question):
    if question == "discount": 
        discount_rate = input("What is the discounted rate? (The standard is 15% since  index founds provide 10% return)\n")
        if check_numeric(discount_rate):
            discount_rate = float(discount_rate)
            discount_rate = discount_rate / 100 
        else:
            discount_rate = 0.20
        return discount_rate
    
    elif question == "margin of safety": 
        margin_safety = int(input("What margin of safety? (Give the answer in percentage)\n"))
        if check_numeric(margin_safety):
            margin_safety = float(margin_safety)
            margin_safety = margin_safety / 100 
        else:
            margin_safety = 0.20
        return margin_safety
    else:
        print("Question not recognized...\n")
        return False
        
        
        

In [14]:
# Add new columns to dataframes used for DCF analysis
# Also calculate: selling multiple, and number of years of supported data

# CF to Capital selling multiple 
df1["Selling Multiple"] = df1["Capital"] / df1["Cash Flows"]
min_selling_mult = df1["Selling Multiple"].min()
avg_selling_mult = df1["Selling Multiple"].mean()

### Number of years 
##num_years = df3.loc[0, 'End'] - df3.loc[0, 'Start']
##df3["Annual Growth"] = annual_growth(df.loc)

/var/folders/pt/c36bh1bd45n7nwwf9sm76bx00000gn/T/ipykernel_30751/847504276.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Selling Multiple"] = df1["Capital"] / df1["Cash Flows"]


In [15]:
# Relevant choices for different degrees (How convserative)
'''
conservative: 20 percent discount rate and 50 percent margin of safety 
average: 15 percent discount rate and 30 percent margin of safety 
not conservative: 10 percent discount rate and 30 percent margin of safety 
'''

if degree == "conservative":
    discount_rate = 0.20
    margin_safety = 0.5
elif degree == "average":
    discount_rate = 0.15
    margin_safety = 0.3
elif degree == "not conservative":
    discount_rate = 0.10
    margin_safety = 0.3 
elif degree == "manual": # This part of the code not written yet
    discount_rate = ask_input("discount")
    margin_safety = ask_input("margin of safety")

    


In [16]:
# Get required variables from user for DCF analysis: Discount rate, growth rate (min and max), margin of safety
# forecasted years, selling multiple
# *** Provide the following answers in percentage

# Discounted rate: We want at least this much percentage return 
# Perhaps set the standard rate to 15%
discount_rate = input("What is the discounted rate? (The standard is 15% since  index founds provide 10% return)")
if check_numeric(discount_rate):
    discount_rate = float(discount_rate)
    discount_rate = discount_rate / 100 
else:
    discount_rate = 0.20
     

# Growth rate: The min and max rate we expect of cash flow growth rate 
# This can be calculated in the future, rather than manually provided 
min_growth =input("What is the min growth rate?")
if check_numeric(min_growth):
    min_growth = float(min_growth)
    min_growth = min_growth / 100 
else:
    min_growth = 0.10

max_growth =input("What is the max growth rate?")
if check_numeric(max_growth):
    max_growth = float(max_growth)
    max_growth = max_growth / 100 
else:
    max_growth = 0.15

# Margin of safety
try:
    margin_safety = int(input("What margin of safety? (Give the answer in percentage)"))
    margin_safety = margin_safety / 100
except ValueError:
    print("Invalid input for Margin of Safety\n !!!!!!!!!!!!!. \n Taking standard value")
    margin_safety = 0.5

# This is used for price to FCF ratio    
try:
    sell_mult = int(input("What multiple is the stock selling w.r.t the cash flow?"))
except ValueError:
    print("Invalid input for price to cash ratio\n !!!!!!!!!!!!. \n Taking standard value")
    sell_mult = 10
 
try:
    num_years = int(input("How many years forward for prediction? (11 is standard)"))
except ValueError:
    print("Invalid input for price to cash ratio\n !!!!!!!!!!!!. \n Taking standard value")
    num_years = 11

print(f"Variable check:\n {discount_rate, min_growth, max_growth, margin_safety, sell_mult, num_years}")

What is the discounted rate? (The standard is 15% since  index founds provide 10% return)15
What is the min growth rate?10
What is the max growth rate?20
What margin of safety? (Give the answer in percentage)50
What multiple is the stock selling w.r.t the cash flow?10
How many years forward for prediction? (11 is standard)11
Variable check:
 (0.15, 0.1, 0.2, 0.5, 10, 11)


In [17]:
# Calculate the lower and upper bound estimate of the intrinsic value

# Variables for intrinsic value range (lower bound and upper bound)
totalLB_CF = 0
totalUB_CF = 0
    
# Get base CF, either choose min cash flow - operating / maintenance costs 
base_CF = cf_min # - operating_costs 
print(f"base cash flow is: {base_CF}")    

# These are present value calculations
    
# Add future cash flows and stock selling price  
for years in range(num_years):
    # Skipping curret year earnings 
    if years == 0:
        continue
    totalLB_CF += (base_CF * pow(1 + min_growth, years)) / pow(1 + discount_rate, years)
    totalUB_CF += (base_CF * pow(1 + max_growth, years)) / pow(1 + discount_rate, years)
    
# Add the terminal cost(from selling the stock at the end of num_years)
totalLB_CF += ((base_CF * pow(1 + min_growth, num_years - 1)) * sell_mult) / pow(1 + discount_rate, num_years - 1)
totalUB_CF += ((base_CF * pow(1 + max_growth, num_years - 1)) * sell_mult) / pow(1 + discount_rate, num_years - 1)

# Add the assets to intrinsic value 
# Filter / Delete the empty values of a column
df_filtered = df.dropna(subset=['Assets'])

for assets in df_filtered["Assets"]:
    totalLB_CF += assets 
    totalUB_CF += assets
    
print("Before Margin of Safety")    
print(f"LB intrinsic value is {totalLB_CF}")
print(f"UB intrinsic value is {totalUB_CF}\n")
    
# Take into consideration margin of safety
totalLB_CF = totalLB_CF * (1 - margin_safety)
totalUB_CF = totalUB_CF * (1 - margin_safety)
    
print("After Margin of Safety")
print(f"LB intrinsic value is {totalLB_CF}")
print(f"UB intrinsic value is {totalUB_CF}")

base cash flow is: 53.09
Before Margin of Safety
LB intrinsic value is 826.5266903622485
UB intrinsic value is 1555.4889292116452

After Margin of Safety
LB intrinsic value is 413.26334518112424
UB intrinsic value is 777.7444646058226


In [ ]:
# Qualitative measures 

score = []

print("Rate the following criterion from 1-10, with 10 being\
the best and 0 being the worst. ")

moat = input("Does this business have a competitive advantages? Does there exist a moat? \
             If the price of the product is suddenly increased, would customers still pay for it? \
             If another competitor offered a similar product, will it be affected? Give a rating, from 1 - 10")
score.append(moat)



management = input("How is the management? Does the manager do as they say? How about the board of directors? \
Do their actions reflect what they say? \
Look at what they promised in the past annual reports. Give a rating, from 1-10")
score.append(management)



worth = input("The business may be great... But is the price worth it? Give a rating from 1-10.")
score.append(worth)


hype = input("Remember: The Hotter they are, the harder they fall. How hyped is the stock? Give a rating from 1-10.")
score.append(hype)


change = input("How willing is the company to embrace change? Is it still stuck in past achievments? What \
popular new products has to company produced? Give a rating from 1-10.")
score.append(change)


debt = input("How much debt does the company have? What is the percentage? Has the company \
survived a recession during the past? Give a rating from 1-10. ")
score.append(debt)


allocation = input("Is capital efficiently allocated? Are earnings retained earnings, or is it the result \
of efficient capital allocation? Give a rating from.  1-10")
score.append(allocations)


customers = input("Does the company have reliable customers? \
Will sudden changes in price easily sway away customers? Give a rating \
from 1-10.")
score.append(customers)


technology = input("Are there high risks of technological \
obsolescence (becoming oudated)? Or does it have abundant growth opportunities. \
Give a rating from 1-10.")
score.append(technology)


present = input("What are the current expectations / prospects? \
Give a rating from 1-10.")
score.append(present)


future = input("What are the future expectations? (Note that \
favoring future expectations over current supported facts is generally stupid.) \
Give a rating from 1-10.")
score.append(future)

survival = input("Has the company survived stock crashes? Does it \
withstand adversity? Does the company do as they say? \
Give a rating from 1-10.")
score.append(survival)

print(f"The average score is {average := (sum(score) / len(score))}")

# Perhaps, we can introduce a different scoring system. Using the 
# average seems inaccurate. Weighted average will probably 
# be better 



In [ ]:
# Other things to remember
# Remember to read fiancial statements backwards. Things that they don't want to expose are often at the end
# If the financial statements are obscure, it probably means the company itself is unhealthy
# Beware of long term / unreasonable options given to managers
# A manager can fix a decent business, but it cannot fix an inherently bad business (fix leaks, but not the boat itself)
# Don't buy an average business for a cheap price 
# The market is a voting machine, not a rational one... 